In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import json
import time
import fitz
import os
import re
import csv
import asyncio
import aiohttp
from concurrent.futures import ThreadPoolExecutor


In [ ]:
def load_json():
    with open('./config2.json','r')as file:
        data=json.load(file)
    return data

def downlod_pdf(urls_data):
    options=webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_experimental_option('prefs',{
        'download.default_directory':'C:\\Users\\RushikeshSuryagandh\\npAssignment\\Major_Project',  # Gives the directory path to store the pdf
        'download.prompt_for_download':False, # s/w doesn't ask user about confirmation of downloading 
        'plugins.always_open_pdf_externally':True # means that PDF files will always be opened in an external PDF viewer instead of within the software or browser itself.

    })
    driver=webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()),options=options)
    
    for url in urls_data: # for each url in urls_data
        # print(url)
        driver.get(url)
        driver.implicitly_wait(2)


def fetch_pdf_files(directory_path):
    pdf_files = []
    
    # Walk through the directory
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            # Check if the file ends with ".pdf"
            if file.endswith('.pdf'):
                # Append the full file path to the list
                pdf_files.append(os.path.join(root, file))
    
    return pdf_files




def extract_data_from_pdf(pdf_file_paths):
    for pdf in pdf_file_paths:
        # print(pdf)
        open_pdf=fitz.open(pdf)
        time.sleep(2)
        text=''
        # print(open_pdf)
        for page_num in range(4):
            text+=open_pdf[page_num].get_text('text')
        # time.sleep(2)
        print(text)
    return text

def extract_data_from_regx(text):
    email_pattern=r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9._%+-]+\.[a-zA-Z]{2,}'
    cin_number_pattern=r'[A-Z]{1}[0-9]{5}[A-Z]{2}[0-9]{4}[A-Z]{3}[0-9]{6}'
    mobile_number_pattern=r'\b(?:\d{3,5}[\s-]?)?[\d]{7,10}\b'
    pan_number_pattern=r'\b[A-Z]{5}[0-9]{4}[A-Z]{1}\b'
    dates_pattern=r'\b\d{2}/\d{2}/\d{4}\b'
    website_pattern=r'\bhttps?://[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}(?:/\S*)?\b|\bwww\.[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}(?:/\S*)?'
    
    emails = re.findall(email_pattern, text)
    cin_numbers = re.findall(cin_number_pattern, text)
    mobile_numbers = re.findall(mobile_number_pattern, text)
    pan_numbers = re.findall(pan_number_pattern, text)
    dates = re.findall(dates_pattern, text)
    websites = re.findall(website_pattern, text)

    # print(emails,cin_numbers,mobile_numbers,pan_numbers,dates,websites)
    data = []
    for i in range(max(len(emails), len(cin_numbers), len(mobile_numbers), len(pan_numbers), len(dates), len(websites))):
        data_row = {
            'Email': emails[i] if i < len(emails) else '',
            'CIN': cin_numbers[i] if i < len(cin_numbers) else '',
            'Mobile Number': mobile_numbers[i] if i < len(mobile_numbers) else '',
            'PAN': pan_numbers[i] if i < len(pan_numbers) else '',
            'Date': dates[i] if i < len(dates) else '',
            'Website': websites[i] if i < len(websites) else ''
        }
        data.append(data_row)
    return data

def save_to_csv(data,file_name='pdf_results.csv') :
    with open(file_name,mode='w',newline='',encoding='utf-8')as file:
        writer = csv.DictWriter(file, fieldnames=['Email', 'CIN', 'Mobile Number', 'PAN', 'Date', 'Website'])
        writer.writeheader()
        for row in data:
            writer.writerow(row)
    print(f'Data Succesfully saved in {file_name}')

    

    

def main():
    config=load_json()
    urls_data=config['urls']
    downlod_pdf(urls_data)
   
    # Specify your directory path
    directory_path = config['directory_path']

    # Fetch the PDF file paths
    pdf_file_paths = fetch_pdf_files(directory_path)

    

    # Print the list of PDF file paths
    # print(pdf_file_paths)
    text=extract_data_from_pdf(pdf_file_paths)
    data=extract_data_from_regx(text)
    save_to_csv(data)
    
    
if __name__=="__main__":
    start_time=time.time()
    main()
    end_time=time.time()
    print(end_time-start_time)
    
    
    


Page 1 of 21
 FORM NO. MGT-7  
[Pursuant to sub-Section(1) of section 92 of 
the Companies Act, 2013 and sub-rule (1) of 
rule 11of the Companies (Management and 
Administration) Rules, 2014]  
   
 
 Annual Return 
(other than OPCs and Small 
Companies) 
   
 
  
   I. REGISTRATION AND OTHER DETAILS  
 
 
  (i) * Corporate Identification Number (CIN) of the company 
L74899HR1995PLC095967
Hindi
English
Form language
Refer the instruction kit for filing the form.
         Global Location Number (GLN) of the company  
  (ii)  (a) Name of the company 
BHARTI AIRTEL LIMITED
  (b) Registered office address
Airtel Center, Plot No. 16, Udyog Vihar, 
Phase-IV, 
Gurugram 
Gurgaon 
Haryana 
122015
        (c) *e-mail ID of the company  
compliance.officer@bharti.in
        (d) *Telephone number with STD code
01146666100
        (e)  Website
www.airtel.com
  (iii)      Date of Incorporation 
07/07/1995
 Type of the Company  
   
 Category of the Company  
   
 Sub-category of the Company  
   
Pu